## Cutting GRID 
Memotong raster dengan sampel besar menjadi beberapa raster yang lebih kecil

#### [00] Load Library

In [1]:
import pandas as pd
import numpy as np
import math

import geopandas as gpd 
import rasterio 
from shapely.geometry import Polygon
from osgeo import gdal

from google.cloud import storage
import gcsfs

from datetime import datetime
import glob
import os
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from omegaconf import DictConfig,OmegaConf




**!! Edit kdPIC parameter in config/config.yaml**

specify the "kdPIC" and the band that would be calculated

In [2]:
conf = OmegaConf.load('config/config.yaml')

In [3]:
kdPIC=conf['config']['pic_']

#### [01] Import Data 

In [4]:
file_sampel = kdPIC + "_sample2022.gpkg"
sampel = gpd.read_file("gs://bps-gcp-bucket/MLST2023/sample/"+file_sampel).to_crs(4326)
grid = gpd.read_file("gs://bps-gcp-bucket/MLST2023/sample/Grid_Ekoreg_2022.gpkg").query('kdPIC=='+'\"'+kdPIC+'\"')

#### [02] Function Definition


**[02.01]** Mendapat nama file raster

In [5]:
def get_name(grid, pic):
    filename= 'duatahun_'+grid+'_QALPN1_PakKus_sentinel2_10m.tif'
    filename_full = '/vsigs/bps-gcp-bucket/citra-sentinel2/'+pic+'/'+filename
    return filename, filename_full


**[02.02]** Mendapat id grid terhadap treshold tertentu

In [6]:
def get_id(sampel, treshold):
    id_ = pd.pivot_table(sampel, index="ID_GRID", aggfunc="count").sort_values(by="geometry",ascending=False).query("geometry>"+str(treshold)).index
    return list(id_)

**[02.03]** Memotong raster grid

In [7]:
def create_grid_raster(id_grid, n): 
    filename, filename_full = get_name(id_grid, kdPIC)
    
    ds = gdal.Open(filename_full)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize

    tile_size_x = math.ceil(xsize/n)
    tile_size_y = math.ceil(ysize/n)

    k = 1
    for i in range(0, xsize, tile_size_x):
        for j in range(0, ysize, tile_size_y):
            ds = gdal.Translate('ml_output/00_cutting_to_subgrid/' + filename[0:9] +  id_grid + "_"  + str(k) + filename[-32:], filename_full, srcWin = [i, j, tile_size_x, tile_size_y])
            ds = None
            k+=1
            

**[02.04]** Memotong vector grid

In [8]:
def create_grid_polygon(id_grid, n): 
    
    tmp = grid.loc[grid.ID_GRID == id_grid]
    xmin,ymin,xmax,ymax =  tmp.total_bounds
    width = (xmax-xmin)/n
    height = (ymax-ymin)/n
    rows = n
    cols = n
    XleftOrigin = xmin
    XrightOrigin = xmin + width
    YtopOrigin = ymax
    YbottomOrigin = ymax- height
    polygons = []

    for i in range(n):
        Ytop = YtopOrigin
        Ybottom =YbottomOrigin
        for j in range(n):
            polygons.append(Polygon([(XleftOrigin, Ytop), (XrightOrigin, Ytop), (XrightOrigin, Ybottom), (XleftOrigin, Ybottom)])) 
            Ytop = Ytop - height
            Ybottom = Ybottom - height
        XleftOrigin = XleftOrigin + width
        XrightOrigin = XrightOrigin + width
    
    sub_grid = gpd.GeoDataFrame({'id_sub':range(1,n**2+1),
                             'geometry':polygons})
    sub_grid['sub_grid'] = [id_grid + '_' + str(x) for x in sub_grid.id_sub]
    
    return sub_grid

#### [03] List of GRID dengan sampel besar

List grid dengan sampel lebih dari 1000 titik sampel

In [9]:
id_grid_ = list(sampel.ID_GRID.unique())
id_ = get_id(sampel, 1000) ## edit dengan maksimum sampel
id_

['ID-3602',
 'ID-3225',
 'ID-3552',
 'ID-3497',
 'ID-3601',
 'ID-3498',
 'ID-3224',
 'ID-3643',
 'ID-3982',
 'ID-3551',
 'ID-3272',
 'ID-3175',
 'ID-3553',
 'ID-4974',
 'ID-3173',
 'ID-3682',
 'ID-4006',
 'ID-3603']

In [10]:
print("Total Seluruh Grid : " + str(len(sampel.ID_GRID.unique())))
print("Total Seluruh Grid dg dengan Sampel > 1000 : " + str(len(id_)))
print("Persentase Seluruh Grid dg dengan Sampel > 1000 : " + str(100*len(id_)/len(sampel.ID_GRID.unique())))


Total Seluruh Grid : 140
Total Seluruh Grid dg dengan Sampel > 1000 : 18
Persentase Seluruh Grid dg dengan Sampel > 1000 : 12.857142857142858


#### [04] Memotong Grid dalam [nxn] Raster

Memotong grid menjadi 25 sub raster

In [11]:
cols_ = list(sampel.columns)
cols_.append('sub_grid')
res = pd.DataFrame(columns=cols_)

for i in tqdm(id_grid_):
    sj = sampel.loc[sampel.ID_GRID == i]
    
    if i in id_:
        create_grid_raster(i, 5) 
        sub_grid = create_grid_polygon(i, 5)
        sj_ = gpd.sjoin(sj, sub_grid[['geometry','sub_grid']])
        res = res.append(sj_)
    else:
        sj['sub_grid'] = i
        res = res.append(sj)


  0%|          | 0/140 [00:00<?, ?it/s]Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/

  7%|▋         | 10/140 [00:24<05:58,  2.76s/it]Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /

 11%|█         | 15/140 [00:48<08:34,  4.11s/it]Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /

Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ:

 21%|██▏       | 30/140 [01:37<06:37,  3.62s/it]Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /

 23%|██▎       | 32/140 [01:59<09:45,  5.42s/it]Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
ERROR 1: PROJ: proj_create_from_name: Open of /

KeyboardInterrupt: 

#### [05] Cek Jumlah Sampel tiap GRID dan SubGrid 

In [12]:
res.groupby(['ID_GRID','sub_grid']).sub_grid.count()

ID_GRID  sub_grid  
ID-3043  ID-3043         20
ID-3044  ID-3044        287
ID-3086  ID-3086        225
ID-3087  ID-3087        631
ID-3128  ID-3128        444
ID-3129  ID-3129        203
ID-3130  ID-3130          5
ID-3173  ID-3173_10       9
         ID-3173_12       8
         ID-3173_13      81
         ID-3173_14       9
         ID-3173_15       8
         ID-3173_16      34
         ID-3173_17     438
         ID-3173_18      72
         ID-3173_19       6
         ID-3173_20       9
         ID-3173_21     137
         ID-3173_22     176
         ID-3173_23      52
         ID-3173_24       7
         ID-3173_25      19
         ID-3173_4      127
         ID-3173_5       27
         ID-3173_8       24
         ID-3173_9       26
ID-3174  ID-3174        515
ID-3175  ID-3175_1        8
         ID-3175_11      31
         ID-3175_12     118
         ID-3175_13      85
         ID-3175_14      58
         ID-3175_16      47
         ID-3175_17      55
         ID-3175_18      73


##### [06] Upload File Raster dan Sampel ke Bucket dalam GCloud Storange

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'config/gcp_store.json'
client = storage.Client()
bucket = client.get_bucket('bps-gcp-bucket')

for i in glob.glob("ml_output/00_cutting_to_subgrid/*.tif"):
    tif_name = i.split('/')[2]
    gcp_file_name='citra-sentinel2/'+kdPIC+'/'+tif_name
    print(gcp_file_name)
    break
    bucket.blob(gcp_file_name).upload_from_filename(i)
    # print(tif_name + " DONE !")
print("DONE !! SELURUH RASTER SUB GRID TELAH TERUPLOAD !")
    
gpd_file_name = "ml_output/00_cutting_to_subgrid/"+file_sampel[:-5]+"_edit.gpkg"
gpd.GeoDataFrame(res, crs="EPSG:4326", geometry="geometry").to_file(gpd_file_name,driver="GPKG")
gcp_file_name=gpd_file_name
bucket.blob("MLST2023/sample/"+kdPIC+"_sample2022_edit.gpkg").upload_from_filename(gcp_file_name)
print("DONE !! FILE GPKG EDIT TELAH TERUPLOAD !")

for i in glob.glob("ml_output/00_cutting_to_subgrid/*.tif"):
    os.remove(i)
    
for i in glob.glob("ml_output/00_cutting_to_subgrid/*.gpkg"):
    os.remove(i)    


citra-sentinel2/F/duatahun_ID-3552_20_QALPN1_PakKus_sentinel2_10m.tif
DONE !! SELURUH RASTER SUB GRID TELAH TERUPLOAD !
